In [1]:
import os
os.chdir("/Users/richcriticism/Documents/Documents/Bhavya_code/CorrectPose2")

In [2]:
import certifi
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
import urllib

In [4]:
%pwd

'/Users/richcriticism/Documents/Documents/Bhavya_code/CorrectPose2'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass
class DataIngestionConfig:
    root_dir : Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.poseClassification.constants import *
from src.poseClassification.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [9]:
from poseClassification import logger
import urllib.request as request
from poseClassification.utils.common import get_size
import zipfile


In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            ssl._create_default_https_context = ssl._create_unverified_context
            filename, headers = urllib.request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-01 14:34:38,053: INFO : common : yaml file : config/config.yaml loaded successfully]
[2025-02-01 14:34:38,055: INFO : common : yaml file : params.yaml loaded successfully]
[2025-02-01 14:34:38,056: INFO : common : creating directory at path: artifacts]
[2025-02-01 14:34:38,057: INFO : common : creating directory at path: artifacts/data_ingestion]
[2025-02-01 14:34:46,847: INFO : 1239877682 : artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 616A:282A8B:6A928:AFD08:679DE3A6
Accept-Ranges: bytes
Date: Sat, 01 Feb 2025 09:04:39 GMT
Via: 1.1 varnish
X-Served-By: cache-